In [3]:
import math
from typing import List, Tuple, Optional

import pandas as pd

Point = Tuple[float, float]

In [4]:
def _ray_dir_from_bearing_deg(bearing_deg: float) -> Tuple[float, float]:
    """
    Bearing in degrees, clockwise from North (the common wind convention).
    Returns a unit direction vector (dx, dy) pointing TOWARD the bearing.
    """
    theta = math.radians(bearing_deg)
    dx = math.sin(theta)  # east component
    dy = math.cos(theta)  # north component
    # already unit length
    return dx, dy

def _ray_segment_intersection(P: Point, R: Point, A: Point, B: Point) -> Optional[float]:
    """
    Ray:  P + t*R, t >= 0   (R should be unit-length for 't' to be distance)
    Segment: A + u*(B-A), u in [0,1]
    Returns t (distance along the ray) if they intersect ahead of P, else None.
    """
    px, py = P
    rx, ry = R
    ax, ay = A
    bx, by = B
    sx, sy = bx - ax, by - ay

    # Solve: P + t*R = A + u*S  ->  [R  -S] [t u]^T = A - P
    denom = rx * (-sy) - ry * (-sx)  # = - (rx*sy - ry*sx)
    if abs(denom) < 1e-12:
        return None  # Parallel or nearly so

    apx, apy = ax - px, ay - py
    t = (apx * (-sy) - apy * (-sx)) / denom
    u = (rx * apy - ry * apx) / denom

    if t >= 0 and 0.0 <= u <= 1.0:
        return t
    return None

def fetch_in_direction(point: Point, polygon: List[Point], bearing_deg: float) -> float:
    """
    Compute fetch (distance to first shoreline intersection) from 'point'
    along the direction 'bearing_deg' (clockwise from North).
    'polygon' must be a closed contour (first == last); if not, it will be treated as closed.
    Returns distance in same units as coordinates. Returns math.inf if no hit (open polygon).
    """
    if len(polygon) < 3:
        raise ValueError("Polygon needs at least 3 distinct vertices.")
    # Treat as closed without duplicating last if already closed
    closed = polygon if polygon[0] == polygon[-1] else polygon + [polygon[0]]
    R = _ray_dir_from_bearing_deg(bearing_deg)

    hits = []
    for i in range(len(closed) - 1):
        A, B = closed[i], closed[i + 1]
        t = _ray_segment_intersection(point, R, A, B)
        if t is not None:
            # If the point lies exactly on the edge and the ray starts on the boundary,
            # we don't want a zero-distance self-hit; skip tiny distances.
            if t > 1e-9:
                hits.append(t)
    return min(hits) if hits else math.inf

def read_shoreline(filename: str):
    coords = []
    with open(filename) as f:
        for line in f:
            if not line.strip():
                continue
            x, y = map(float, line.split())
            coords.append((x, y))
    # close polygon if needed
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    return coords

In [10]:
shoreline_path = r"C:\Users\leroquan\Documents\Data\mitgcm_grids\geneva\shoreline.txt"
point_coord = (540292.98, 150176.08)

input_folder_name = "geneva_june_2023"
point_name = 'lexplore'
output_path = rf"..//runs//{input_folder_name}//basin-length_{point_name}.fet"

In [11]:
poly_shore = read_shoreline(shoreline_path)

In [12]:
print("Fetch due North (0°):", fetch_in_direction(point_coord, poly_shore, 0))
print("Fetch due East (90°):", fetch_in_direction(point_coord, poly_shore, 90))
print("Fetch due South (180°):", fetch_in_direction(point_coord, poly_shore, 180))
print("Fetch due West (270°):", fetch_in_direction(point_coord, poly_shore, 270))

Fetch due North (0°): 593.6951304347965
Fetch due East (90°): 2336.195384615412
Fetch due South (180°): 10614.079999999985
Fetch due West (270°): 13577.225365853652


In [15]:
results = []
for ang in range(0,360):
    length_1 = fetch_in_direction(point_coord, poly_shore, ang)
    length_2 = fetch_in_direction(point_coord, poly_shore, ang+180)
    results.append((ang, length_1+length_2))

In [16]:
# Write .fet file
with open(output_path, "w") as f:
    f.write("angle (°)\tLength (m)\n")
    for ang, length in results:
        f.write(f"{ang}\t{length:.0f}\n")